## Part 3, 25 points (find second best sequence)

### Required helper functions to get all the parameters needed for our modified viterbi algorithm

In [3]:
def process_file(filepath):
    # we make use of the default library "collections" to make processing the tags and word-tag pairs easier
    import collections #used for counting
    tag_count = collections.defaultdict(int)  # counting for tags
    word_tag_count = collections.defaultdict(int)  # counting for word-tag pairs
    vocabulary = set()  # stores unique words
    sentences = []
    current_sentence = []

    with open(filepath, 'r', encoding='utf-8') as file:
        # reading file line-by-line
        for line in file:
            stripped_line = line.strip() #removes the /n and then splits it to separete the word and its label
            if stripped_line:  # check if there even is a word or tag in the line
                word, tag = stripped_line.split()  # Split line into word and tag
                word_tag_count[(word, tag)] += 1
                tag_count[tag] += 1
                vocabulary.add(word) #doesnt add duplicates
                current_sentence.append(word)
            else:
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
        if current_sentence:
            sentences.append(current_sentence)

    return tag_count, word_tag_count, vocabulary, sentences


#tag count : dictionary with the count of each tag e.g ('B-NP') : 45
#word_tag_count : dictionary with the count of each word-tag pair e.g ('Municipal','B-NP') : 1

tag_count, word_tag_count, vocabulary, sentences = process_file('EN/train')


In [4]:
def process_file_for_transitions(filepath):
    # we make use of the default library "collections" to make processing the tags and word-tag pairs easier
    import collections
    transition_count = collections.defaultdict(int) #y_u to y_v, including start and stop
    tag_count = collections.defaultdict(int)  # counting for tags
    vocab = set()
    
    # we still need counters for stop and start, to add them into the transition parameters
    start_counter = 0 
    stop_counter = 0
    
    START = "START"
    STOP = "STOP"
    previous_tag = START

    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            stripped_line = line.strip()
            if stripped_line:
                word, tag = stripped_line.split()
                transition_count[(previous_tag, tag)] += 1
                if previous_tag == "START":
                    start_counter += 1
                tag_count[tag] += 1
                previous_tag = tag
                vocab.add(word)
            else:  # when the sentence has ended
                transition_count[(previous_tag, STOP)] += 1
                stop_counter += 1
                previous_tag = START  # reset for the next sentence
     #adding counts for start and stop
    tag_count["START"] = start_counter
    tag_count["STOP"] = stop_counter
    
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read().strip()
    # split on double newlines which denote separated sentences in our case
    sentences = [sentence.split() for sentence in content.split('\n\n')]

    return transition_count, tag_count, sentences, vocab

In [5]:
def get_sentences(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read().strip()
    # split on double newlines which denote separated sentences in our case
    return [sentence.split() for sentence in content.split('\n\n')]


In [6]:
# run the functions 
transition_count, tag_count, sentences, vocabulary = process_file_for_transitions('EN/train')

In [18]:
# we need to use viterbi to predict sentence by sentence
def get_prediction(filepath, tag_count, transition_probabilities, emission_probabilities, vocabulary):
    sentences = get_sentences(filepath)
    predictions = [] #initialise the list of sentences
    for sentence in sentences:
       
        # predict the best path, sentence by sentence
        best_path_prediction = modified_viterbi_algorithm(sentence, tag_count, transition_probabilities, emission_probabilities, vocabulary)
       
        #puts the word - predicted tag pairs in the predictions array pairwise
        predictions.append(list(zip(sentence, best_path_prediction))) 
        
    return predictions
    

In [19]:
def write_tag_predictions_to_file(predictions, output_filepath):
    # open the output file for writing
    with open(output_filepath, 'w', encoding='utf-8') as file:
        for sentence in predictions:
            for word, tag in sentence:
                # write each word and its predicted tag to the file, with a spacing to separate them.
                file.write(f"{word} {tag}\n")
            # leave an empty line between sentences
            file.write("\n")

In [20]:
def estimate_all_emission_probabilities_with_unknown(tag_count, word_tag_count, k =0.1):
  
    emission_probabilities = {}
    # iterate through all the word tag pairs to get all the emission probabilities
    # store the results in the dictionary emission_probabilities
    for (word, tag), count in word_tag_count.items():
        
        emission_probabilities[(word, tag)] = count / (tag_count[tag]+k)
        
    for tag, count in tag_count.items():
        emission_probabilities[("#UNK#", tag)] = count / (tag_count[tag]+k)
    return emission_probabilities

In [21]:
emission_probabilities = estimate_all_emission_probabilities_with_unknown(tag_count, word_tag_count)


In [22]:
def estimate_all_transition_probability(transition_count, tag_count):
  
    transition_probabilities = {}
    # iterate through all the transition tag pairs to get all the transition probabilities
    # store the results in the dictionary transition_probabilities
    for (y_u, y_v), count in transition_count.items():
        transition_probabilities[(y_u, y_v)] = count / tag_count[y_u]
        
    return transition_probabilities

In [23]:
# run the function to get all the transiiton probaibilities
transition_probabilities = estimate_all_transition_probability(transition_count, tag_count)

## Our approach to modifying the viterbi algorithm to obtain the second best path includes:

In [39]:
def modified_viterbi_algorithm(sentence, tag_count, transmission_probabilities, emission_probabilities, vocabulary, unk = 0.1):
    # make sure that "sentence" is a sequence of x observations
    tags = [tag for tag in tag_count if tag not in ['START', 'STOP']] # makes a dictionary of tags that doesnt include start and stop, so we dont iterate through them unncessarily
#     tags.pop('START')
#     tags.pop('STOP')
    
    n = len(sentence)  # number of words in the sentence (k)
    m = len(tags)      # number of tags (u / v)
    for i in range(0,n):
        if sentence[i] not in vocabulary:
            sentence[i] = '#UNK#'
    # create a matrix to store all the pi values
    pi = [[float('-inf')] * m for _ in range(n+1)] #+1 tp account for the stop state, but we dont actually store anything theer
    backpointer = [[[0,0]] * m for _ in range(n)] # to store y*

    # base case!!!, here, we actually just initialise the first column to be pi(0,v) where u is "START". 
    # we skip the step of assigning pi(0,v) = 1 if v is start and 0 otherwise
    for i, tag in enumerate(tags): # i is the index of the tag
        t_count = tag_count[tag]
        pi[0][i] = transmission_probabilities.get(('START', tag), 0) * emission_probabilities.get((sentence[0], tag),0)

            
    #bottom up dynamic programming        
    for i in range(1, n): 
        for j, tag in enumerate(tags):
            max_prob = float('-inf')
            max_state = None
            prev_max_state = None

            for kk, prev_tag in enumerate(tags):
                # for now, if there's an unknown word, default to 0, but we should change this to the k/count + k thing
#                 t_count
                

                prob = pi[i-1][kk] * transmission_probabilities.get((prev_tag, tag), 0) * emission_probabilities.get((sentence[i], tag), 0)
                # We are going through every tag that could have existed at the previous state, and finding the maximum probability
                # of transitioning from that tag to the current tag, and multiplying it by the emission probability of the current word

                # if the probability is greater than the max probability, update the max probability and the max state
                # max state is the tag that gave us the maximum probability of transitioning to the current tag
                # max probability is the maximum probability of transitioning to the current tag
                
                if prob > max_prob:
                    max_prob = prob
                    prev_max_state = max_state
                    max_state = kk

            pi[i][j] = max_prob
            backpointer[i][j][0] = max_state # to store y*
            backpointer[i][j][1] = prev_max_state

            #backpointer contains the best state that led to the current state
            #backpointer[i][j] = k means that the state that led to the current state is k

    # termination step 
    max_prob = float('-inf')
    max_state = None
    # we want to find the maximum probability of transitioning from the last word to the stop state



    for i, tag in enumerate(tags):
        prob = pi[n-1][i] * transmission_probabilities.get((tag, 'STOP'), 0) # we dont need to multiply by the emission probability of the stop state because it is always 1
        # We are going through every tag at n-1 time stamp (time stamp before 'STOP')
        # To see which state has the best chance to lead to a STOP state

        if prob > max_prob:
            max_prob = prob # Probability of the max state leading to stop
            max_state = i # Most probable state to lead to stop
    
    
            
    #initialise an array for the best sequence of states         

    second_best = [tags[max_state]]

    second_best_explored = False
    #go backwards along the backpointer, iteratively finding the best state
    for i in range(n-1, 0, -1): # n is going down by -1, we're going down time stamps from n-1 to 1

        max_state = backpointer[i][max_state][0] # get the best state that led to the current state


        if backpointer[i][max_state][1] != None and second_best_explored == False: # next best path exists and that we havent explored it yet
            second_best.append(tags[backpointer[i][max_state][1]])

            second_best_explored = True

        else:

            second_best.append(tags[backpointer[i][max_state][0]])

    # reverse the array to get it in the right order
    second_best.reverse()
    
    return second_best

predictions= get_prediction('EN/dev.in', tag_count, transition_probabilities, emission_probabilities, vocabulary)
# print(predictions)

write_tag_predictions_to_file(predictions, 'EN/dev.p3.out')

!python3 EvalScript/evalResult.py EN/dev.out EN/dev.p3.out





#Entity in gold data: 13179
#Entity in prediction: 24725

#Correct Entity : 7231
Entity  precision: 0.2925
Entity  recall: 0.5487
Entity  F: 0.3815

#Correct Sentiment : 2114
Sentiment  precision: 0.0855
Sentiment  recall: 0.1604
Sentiment  F: 0.1115
